In [1]:
import sys
sys.path.append('../')

from activation import *
from loss import *
from linear import * 
from encapsulage import *
from utils import *

import matplotlib.pyplot as plt
import random
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
import seaborn as sns
from sklearn.metrics import confusion_matrix

def onehot(x): 
    out = [0] * 10
    out[x] = 1
    return out

In [3]:
# Charger les données MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalisation
X_train /= 255
X_test /= 255

# One-hot encoding
X_train = X_train[:1000]
y_train = y_train[:1000]
X_test = X_test[:100]
y_test = y_test[:100]
#y_train = np.array([onehot(x) for x in y_train])
#y_test = np.array([onehot(x) for x in y_test])  



# Paramètres
# d = X_train.shape[1]
# batchsize = 200
# gradient_step = 1e-5


# hidden_dim = 128
# hidden_dim2 = 64
# out_dim = 10
# epoch = 1000



# lin = Linear(d, hidden_dim)
# lin2 = Linear(hidden_dim, hidden_dim2)
# lin3 = Linear(hidden_dim2, hidden_dim)
# lin4 = Linear(hidden_dim, out_dim)
# lin3._parameters = lin2._parameters.T
# lin4._parameters = lin._parameters.T

# encodeur = [lin, TanH(), lin2, TanH()]
# decodeur = [lin3, TanH(), lin4, Sigmoid()]

# net = Sequentiel(encodeur + decodeur)


# paramétrer le modèle
d = X_train.shape[1]
print(d)
d_prime = 196
d_prime2 = 100
out = 784
epoch = 10000
gradient_step = 1e-5
batchsize = 100
lin = Linear(d, d_prime)
lin2 = Linear(d_prime, d_prime2)
lin3 = Linear(d_prime2, d_prime)
lin4 = Linear(d_prime, out)
lin3._parameters = lin2._parameters.T
lin4._parameters = lin._parameters.T
tan = TanH()
sig = Sigmoid()
loss = MSELoss()

Encodeur = [lin,tan,lin2,tan]
Decodeur = [lin3,tan,lin4,sig]

net = Sequentiel(Encodeur + Decodeur)


# # Perte
# loss = BCELoss()

# Entraînement
optim = Optim(net, loss, gradient_step)
lloss = optim.SGD(X_train, y_train, batchsize, epoch)


784


  0%|          | 0/10000 [00:00<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (100,) (100,784) 

In [ ]:
# Affichage de la perte
plt.figure()
plt.plot(lloss)
plt.title("Perte au cours des époques")
plt.xlabel("Époques")
plt.ylabel("Perte")
plt.grid()